In [3]:
#Initial imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path

%matplotlib inline

## Data Preprocessing

RNN input requires an array data type. The `full_review_text` column will be transformed into the `X` array and the “sentiment” column into the `y` array.

In [4]:
#Import Data
df = pd.read_csv('Housing_values.csv')
#test = pd.read_csv("/data/Housing/test.csv")
df.head()

,Identifier,RegionID,Bedroom,RegionName,RegionType,StateName,State,Metro,CountyName,1/31/2010,...,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020
0,2,39051,1,Houston,City,TX,TX,Houston-The Woodlands-Sugar Land,Harris County,67653.0,...,104937,105155,105502,105618,106083,106186,106299,106161,106271,106635
1,110,7204,1,Spring,City,TX,TX,Houston-The Woodlands-Sugar Land,Harris County,54159.0,...,88793,89402,90151,90444,91064,91668,92500,93108,93931,94948
2,187,41114,1,Sugar Land,City,TX,TX,Houston-The Woodlands-Sugar Land,Fort Bend County,212721.0,...,239912,239079,239124,239166,240193,242007,241984,241888,242018,241986
3,209,38119,1,Cypress,City,TX,TX,Houston-The Woodlands-Sugar Land,Harris County,83279.0,...,134336,135568,135976,136333,137155,137673,138310,137995,137454,136950
4,218,44734,1,Conroe,City,TX,TX,Houston-The Woodlands-Sugar Land,Montgomery County,142717.0,...,220807,221831,222446,223048,224432,224717,224159,222976,222007,220157


In [5]:
#slice data into Training and Testing
train = df.loc['2010':'2019']

test = df.loc['2020']

KeyError: '2020'

To train the RNN model, we need to encode the text data as an integer. This transformation can be done using the following tools from Keras.

In [11]:
#importing function
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def factorize(data, var, fill_na = None):
      if fill_na is not None:
            data[var].fillna(fill_na, inplace=True)
      le.fit(data[var])
      data[var] = le.transform(data[var])
      return data

In [4]:
# Import Keras modules for data encoding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
# Create an instance of the Tokenizer and fit it with the X text data
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(X)

In [8]:
# Padding sequences
X_pad = pad_sequences(X_seq, maxlen=140, padding="post")

Now that the data is encoded, the training and testing sets will be created.

In [9]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pad, y, random_state=78)

In [10]:
# Import Keras modules for model creation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

 ### Making Predictions